# Canadian Energy Solutions

This is a simple notebook that allows us to explore some of the underlying problems in the Canadian energy space. The objective is to identify sectors that need assistance, and potentially identify companies that can fill those gaps.

If we were to simplify the Canadian energy problem, we can say that we need to reach a scenario where two simple constraints are met:

`supply > demand && net_emissions <= 0`

In reality, this is a much more complex equation since if we were to sum the entirety of Canada's production, and subtract demand, we would be glossing over regional differences and distances.

For the sake of this document, we will start by assuming two things:
- A province must be fully sufficient. That is to say that it must not depend on neighbouring provinces to provide electricity.
- Emissions across the country must be net. One province may pollute, as long as neighbouring provinces can assist with the reduction (it's all one atmosphere.)

In [99]:
import numpy as np
import pandas as pd

# Data Import

This primarily uses Canada's Energy Futures 2020 Dataset. This dataset looks far into the future to give us a good idea of whether we're heading int he right direction or not. Unfortunately, there are missing pieces. Notably, uranium data, which is pulled from a variety of other sources.

>  The Energy Futures series explores how possible energy futures might unfold for Canadians over the long term.
>  Canada’s Energy Future 2020: Energy Supply and Demand Projections to 2050 (EF2020) is our latest long-term energy outlook.
>  It is the first outlook in the series to provide projections to 2050.
>  It covers all energy commodities, and all provinces and territories.
>  We use economic and energy models to develop this outlook. We also make assumptions about technology,
>  energy and climate policies, energy markets, human behaviour and the economy.

https://open.canada.ca/data/en/dataset/bba41250-261a-4f3b-9ce8-db44d9a0f725

Could also pull from: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2510007901&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2020&cubeTimeFrame.endMonth=12&cubeTimeFrame.endYear=2020&referencePeriods=20200101%2C20201201

In [100]:
# Butane
print("Importing Butane Data...")
butane_data = pd.read_csv("data/cer-butanes-2020.csv")

# Coal
print("Importing Coal Data...")
coal_data = pd.read_csv("data/cer-coal-2020.csv")

# Crude Oil
print("Importing Crude Oil Data...")
crude_oil_production_data = pd.read_csv("data/cer-crude-oil-production-2020.csv")

print("Importing Crude Oil Export Data...")
crude_oil_export_data = pd.read_csv("data/cer-crude-oil-exports-by_type-annual.csv")

# Electricity Generation
print("Importing Electricity Generation Data...")
electricity_data = pd.read_csv("data/cer-electricity-generation-2020.csv")
electricity_interchange_data = pd.read_csv("data/cer-electricity-interchange-2020.csv", encoding="ISO-8859-1")

statscan_electricity_data_2020 = pd.read_csv("data/statscan-electric-power-generation-2020.csv")

# Ethane
print("Importing Ethane Data...")
ethane_data = pd.read_csv("data/cer-ethane-2020.csv")

# Natural Gas Production
print("Importing Natural Gas Data...")
natural_gas_data = pd.read_csv("data/cer-natural-gas-production-2020.csv")

print("Importing Natural Gas Export Data...")
natural_gas_export_data = pd.read_csv("data/cer-natural-gas-exports-and-imports-annual.csv")

# Pentanes
print("Importing Pentanes Data...")
pentanes_data = pd.read_csv("data/cer-pentanes-2020.csv")

# Propane
print("Importing Propane Data...")
propane_data = pd.read_csv("data/cer-propane-2020.csv")

# Overall Import/Export Data
print("Importing CER Imports/Exports Data...")
cer_imports_exports = pd.read_csv("data/CER_imports_exports_data.csv")

# Primary Demand
print("Importing Primary Demand...")
primary_demand_data = pd.read_csv("data/cer-primary-energy-demand-2020.csv")

# End Use Demand
print("Importing End Use Demand Data...")
end_use_demand_data = pd.read_csv("data/cer-end-use-demand-2020.csv")

# TODO: Secondary Energy Use Database
# https://oee.nrcan.gc.ca/corporate/statistics/neud/dpa/data_e/databases.cfm?attr=0

print("Done!")

Importing Butane Data...
Importing Coal Data...
Importing Crude Oil Data...
Importing Crude Oil Export Data...
Importing Electricity Generation Data...
Importing Ethane Data...
Importing Natural Gas Data...
Importing Natural Gas Export Data...
Importing Pentanes Data...
Importing Propane Data...
Importing CER Imports/Exports Data...
Importing Primary Demand...
Importing End Use Demand Data...
Done!


In [101]:
# Data setup
layer_1_production_and_imports = {}
layer_2_primary_energy = {}
layer_3_industry = {}

In [102]:
# Helper Methods
# Conversion Data: https://apps.cer-rec.gc.ca/Conversion/conversion-tables.aspx?GoCTemplateCulture=en-CA

# There are 1,000,000 GJ in one PJ
gj_to_pj = 1000000

# Onw MWh is 0.000036 PJ
mwh_to_pj = 0.0000036

def gigajoule_to_quad(gj):
  return -1

# Butane, in Cubic Metres
def butane_to_pj(butane):
  return butane * 28.62 / gj_to_pj

# Coal, in tons
# Coal has four different "types" each resulting in different energy densities.
#   Anthracite: 27.70 GJ
#   Bituminous: 27.60 GJ
#   Lignite: 14.40 GJ
#   Subbituminous: 18.80 GJ
def coal_to_pj(coal):
  return coal * 27.60 / gj_to_pj

# Light Crude Oil, in m³
#   Light	1.0 Cubic metres (m³)	38.51 Gigajoules (GJ)
def light_crude_oil_to_pj(crude_oil):
  return crude_oil * 38.41 / gj_to_pj

# Heavy Crude Oil, in m³
#   Heavy	1.0 Cubic metres (m³)	40.90 Gigajoules (GJ)
def heavy_crude_oil_to_pj(crude_oil):
  return crude_oil * 40.90 / gj_to_pj

# C5+, in m³
#   Pentanes plus	1.0 Cubic metres (m³)	35.17 Gigajoules (GJ)
def pentanes_plus_crude_oil_to_pj(crude_oil):
  return crude_oil * 35.17 / gj_to_pj

# Synthetic Crude Oil, in m³
#   Synthetic crude oil	1.0 Cubic metres (m³)	39.40 Gigajoules (GJ)
def synthetic_crude_oil_to_pj(crude_oil):
  return crude_oil * 39.40 / gj_to_pj

# Bitument Crude Oil
#   Bitumen	1.0 Cubic metres (m³)	42.80 Gigajoules (GJ)
def bitumen_crude_oil_to_pj(crude_oil):
  return crude_oil * 42.80 / gj_to_pj

# Ethane, in Cubic Metres
def ethane_to_pj(ethane):
  return ethane * 18.36 / gj_to_pj

# Natural Gas, in Cubic Metres
def natural_gas_to_pj(natural_gas):
  return natural_gas * 0.0373 / gj_to_pj

def pentanes_to_pj(pentanes):
  return pentanes * 35.17 / gj_to_pj

def propane_to_pj(propane):
  return propane * 25.53 / gj_to_pj

# Using https://www.cesarnet.ca/visualization/sankey-diagrams-canadas-energy-systems 's conversion,
# > The primary heat energy generated by uranium depends on the reactor technology being used, and is much higher in reactors that use enriched uranium than in the CANDU natural uranium reactors used in Canada. The typical heat rate for uranium in a CANDU is 7700 MW-days(thermal)/tonne U, which converts to 0.665 PJ/tonne U, and this is the the conversion factor used in this Sankey diagram for estimating the primary energy content of uranium.
def uranium_to_pj(uranium):
  return uranium * 0.665

def gwh_to_pj(gwh):
  # 1 GWh = 3600 GigaJoules
  return gwh * 3600 / gj_to_pj

def extract_value(dataframe, variable, key="Value"):
  if isinstance(variable, list):
    return dataframe[dataframe["Variable"].isin(variable)][key].sum()
  return dataframe[dataframe["Variable"] == variable].iloc[0][key]

In [103]:
def extract_butane_value(data, variable):
  return butane_to_pj(1000 * 365 * extract_value(data, variable))

def extract_butane_data(layer_1, data, year):
  butane_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  # Butane data is "per day". We assume 365 day averages.
  layer_1["butane_imports"] = extract_butane_value(butane_data, "Imports to AB from U.S.")
  layer_1["butane_from_oil_sands"] = extract_butane_value(butane_data, "Production from Oil Sands off-gas")
  layer_1["butane_production"] = extract_butane_value(butane_data, ["Produciton from Refineries", "Production from Gas Processing", "Production from Oil Sands off-gas"])
  layer_1["butane_exports"] = extract_butane_value(butane_data, "Projected Exports")

  layer_1["butane_oil_related_disposition"] = extract_butane_value(butane_data, ["Diluent Demand", "AB Petrochemical Demand", "ON Petrochemical Demand", "Refinery Demand", "Oil Sands Solvent Demand"])
  layer_1["butane_canada_other_disposition"] = extract_butane_value(butane_data, "Canada Other Demand")

  layer_1["butane_adjustments"] = extract_butane_value(butane_data, "Total Supply - Total Disposition")

In [104]:
def extract_coal_data(layer_1, data, year):
  coal_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year)]

  # These values are all in kilotons
  layer_1["coal_imports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Imports")].iloc[0]["Value"])
  layer_1["coal_production"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Production")].iloc[0]["Value"])
  layer_1["coal_exports"] = coal_to_pj(1000 * coal_data[(coal_data["Variable"] == "Total") & (coal_data["Type"] == "Exports")].iloc[0]["Value"])

  #TODO: Use Domestic Demand for other layers

In [105]:
def extract_crude_oil_data(layer_1, data, export_data, import_export_data, year):
  crude_oil_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Thousand Cubic Metres per day")]
  crude_oil_export_data = export_data[(export_data["Year"] == year)]

  #TODO: THis data set is lacking 2020Q4 so we can't do an accurate picture o 2020 imports.
  crude_oil_import_data = import_export_data[(import_export_data["Period"].isin([f'{year}Q1', f'{year}Q2', f'{year}Q3', f'{year}Q4'])) & (import_export_data["Product"] == "Crude Oil") & (import_export_data["Activity"] == "Imports") & (import_export_data["Value"] != "Confidential")]

  # We're missing data from 2020Q4, so take the mean of Q1, Q2, and Q3
  crude_oil_import_data = import_export_data[(import_export_data["Product"] == "Crude Oil") & (import_export_data["Activity"] == "Imports") & (import_export_data["Value"] != "Confidential")]

  import_q1 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q1']["Value"].astype(float).sum()
  import_q2 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q2']["Value"].astype(float).sum()
  import_q3 = crude_oil_import_data[crude_oil_import_data["Period"] == f'{year}Q3']["Value"].astype(float).sum()

  # TODO: Get real data. Average is not a great representation.
  # TODO: We aren't told what kind of crude oil is imported (marked as N/A) therefore this is a guess and we just label it as heavy as a "best effort".
  average_import = (import_q1 + import_q2 + import_q3) / 3
  layer_1["crude_oil_imports"] = heavy_crude_oil_to_pj(1000 * 365 * average_import)

  light_crude_oil = extract_value(crude_oil_data, "Conventional Light", "value")
  heavy_crude_oil = extract_value(crude_oil_data, "Conventional Heavy", "value")
  pentanes_plus_crude_oil = extract_value(crude_oil_data, "C5+", "value")
  synthetic_crude_oil = extract_value(crude_oil_data, "(Upgraded Bitumen)", "value")
  bitumen_crude_oil = extract_value(crude_oil_data, "Mined Bitumen", "value") + extract_value(crude_oil_data, "In Situ Bitumen", "value") - synthetic_crude_oil
  field_condensate_crude_oil = extract_value(crude_oil_data, "Field Condensate", "value")
  layer_1["crude_oil_field_condensate"] = butane_to_pj(1000 * 365 * field_condensate_crude_oil)
  layer_1["crude_oil_production"] = heavy_crude_oil_to_pj(1000 * 365 * heavy_crude_oil) + light_crude_oil_to_pj(1000 * 365 * light_crude_oil) + pentanes_plus_crude_oil_to_pj(1000 * 365 * pentanes_plus_crude_oil) + bitumen_crude_oil_to_pj(1000 * 365 * bitumen_crude_oil) + synthetic_crude_oil_to_pj(1000 * 365 * synthetic_crude_oil)

  light_and_medium_exports = 365 * crude_oil_export_data[crude_oil_export_data["Oil Type"].isin(["Light", "Medium"])]["Volume (m3/d)"].sum()
  heavy_exports = 365 * crude_oil_export_data[crude_oil_export_data["Oil Type"] == "Heavy"]["Volume (m3/d)"].iloc[0]

  layer_1["crude_oil_exports"] = light_crude_oil_to_pj(light_and_medium_exports) + heavy_crude_oil_to_pj(heavy_exports)

## Electricity Data
We pull Electricity Data from 3 sources (all from CER)

1. Primary Energy Demand
2. Electricity Generation
3. End Use Energy Demand

### Primary Demand
> Primary demand is calculated by adding the energy used to generate electricity to total end-use demand, and then subtracting the end use demand for electricity. Removing end-use electricity demand from the total is necessary to avoid double counting.

> This value of primary natural gas demand is higher than the domestic demand value shown in Figure 6.5 because it includes non-marketed natural gas used directly by those that produce it. Examples of this include flared gas, natural gas produced and consumed by in situ oil sands producers, and natural gas produced and consumed by offshore oil production.

> The difference in energy use by fuel for power generation and power generation by fuel in the electricity tables reflects heat losses. Hydro, solar and wind energy use in this table is calculated by directly converting the generation values to petajoules.

### Electricity Generation
This is electricity generation by fuel type (or by renewable source).

### End Use Energy Demand
Known as secondary demand, this is sector-specific use.


In [106]:
def extract_electricity_data(layer_2, data, primary_demand_data, interchange_data, year):
  electricity_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada")]
  electric_generation_primary_demand_data = primary_demand_data[(primary_demand_data["Scenario"] == "Reference") & (primary_demand_data["Year"] == year) & (primary_demand_data["Region"] == "Canada") & (primary_demand_data["Sector"] == "Electric Generation")]
  electricity_interchange_data = interchange_data[(interchange_data["Scenario"] == "Reference") & (interchange_data["Year"] == year) & (interchange_data["Region"] == "Canada")]
  
  # Gather the Primary Energy Requirements for Electricity Generation (what we used)
  # Values are in PJ
  layer_2["coke_and_coal_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "Coal, Coke and Coke Oven Gas")
  layer_2["hydro_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "Hydro")
  layer_2["natural_gas_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "Natural Gas")
  layer_2["uranium_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "Nuclear")
  layer_2["oil_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "RPP")
  layer_2["other_renewables_electric_generation_demand"] = extract_value(electric_generation_primary_demand_data, "Other Renewables and Landfill Gas")

  # Gather the Electricity Generation end values (what we produced)
  # Values in GW/h
  layer_2["coal_and_coke"] = gwh_to_pj(extract_value(electricity_data, "Coal & Coke"))
  layer_2["hydro_wave_tidal"] = gwh_to_pj(extract_value(electricity_data, "Hydro / Wave / Tidal"))
  layer_2["natural_gas"] = gwh_to_pj(extract_value(electricity_data, "Natural Gas"))
  layer_2["uranium"] = gwh_to_pj(extract_value(electricity_data, "Uranium"))
  layer_2["oil"] = gwh_to_pj(extract_value(electricity_data, "Oil"))
  # These three comprise of the "other renewables"
  layer_2["biomass_geothermal"] = gwh_to_pj(extract_value(electricity_data, "Biomass / Geothermal"))
  layer_2["solar"] = gwh_to_pj(extract_value(electricity_data, "Solar"))
  layer_2["wind"] = gwh_to_pj(extract_value(electricity_data, "Wind"))
  layer_2["total_electricity_generation"] = layer_2["coal_and_coke"] + layer_2["hydro_wave_tidal"] + layer_2["natural_gas"] + layer_2["uranium"] + layer_2["oil"] + layer_2["biomass_geothermal"] + layer_2["solar"] + layer_2["wind"]

  # Calculate losses
  layer_2["coke_and_coal_loss"] = layer_2["coke_and_coal_electric_generation_demand"] - layer_2["coal_and_coke"]
  layer_2["hydro_loss"] = layer_2["hydro_electric_generation_demand"] - layer_2["hydro_wave_tidal"]
  layer_2["natural_gas_loss"] = layer_2["natural_gas_electric_generation_demand"] - layer_2["natural_gas"]
  layer_2["uranium_loss"] = layer_2["uranium_electric_generation_demand"] - layer_2["uranium"]
  #TODO: Not sure if this loss should be strictkly applied to 'Other Renewables'
  layer_2["other_renewables_loss"] = layer_2["other_renewables_electric_generation_demand"] - layer_2["biomass_geothermal"] - layer_2["solar"] - layer_2["wind"]
  layer_2["oil_loss"] = layer_2["oil_electric_generation_demand"] - layer_2["oil"]

  layer_2["electricity_imports"] = gwh_to_pj(extract_value(electricity_interchange_data, "Imports"))
  layer_2["electricity_exports"] = gwh_to_pj(extract_value(electricity_interchange_data, "Exports"))

  layer_2["total_electricity_primary_demand"] = layer_2["coke_and_coal_electric_generation_demand"] + layer_2["hydro_electric_generation_demand"] + layer_2["natural_gas_electric_generation_demand"] + layer_2["uranium_electric_generation_demand"] + layer_2["other_renewables_electric_generation_demand"] + layer_2["oil_electric_generation_demand"]

  layer_2["total_electricity_losses"] = layer_2["coke_and_coal_loss"] + layer_2["hydro_loss"] + layer_2["natural_gas_loss"] + layer_2["uranium_loss"] + layer_2["other_renewables_loss"] + layer_2["oil_loss"]

In [107]:
def extract_ethane_value(data, variable):
  return ethane_to_pj(1000 * 365 * extract_value(data, variable))

def extract_ethane_data(layer_1, data, year):
  ethane_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["ethane_imports"] = extract_ethane_value(ethane_data, ["Imports to AB from U.S.", "Imports to ON from U.S."])
  layer_1["ethane_from_oil_sands"] = extract_ethane_value(ethane_data, "Production from Oil Sands off-gas")
  layer_1["ethane_production"] = extract_ethane_value(ethane_data, ["Production from Gas Processing", "Production from Oil Sands off-gas"])
  layer_1["ethane_exports"] = extract_ethane_value(ethane_data, "Exports")

  layer_1["ethane_oil_related_disposition"] = extract_ethane_value(ethane_data, ["AB Petrochemical Demand", "ON Petrochemical Demand", "Oil Sands Solvent Demand"])

  layer_1["ethane_adjustments"] = extract_ethane_value(ethane_data, "Total Supply - Total Disposition")

In [108]:
def extract_natural_gas_data(layer_1, data, export_data, year):
  natural_gas_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada") & (data["Unit"] == "Million Cubic Metres per day")]
  natural_gas_export_data = export_data[(export_data["Port"] == "Total") & (export_data["Year"] == year)]

  #TODO: Confirm http://www.cer-rec.gc.ca/en/data-analysis/energy-commodities/natural-gas/report/natural-gas-summary/natural-gas-annual-trade-summary.html
  layer_1["natural_gas_imports"] = natural_gas_to_pj(1000 * natural_gas_export_data[natural_gas_export_data["Activity"] == "Imports"]["Volume (10^3 m3)"].sum())
  layer_1["natural_gas_production"] = natural_gas_to_pj(1000000 * 365 * natural_gas_data.iloc[0]["Value"])
  layer_1["natural_gas_exports"] = natural_gas_to_pj(1000 * natural_gas_export_data[natural_gas_export_data["Activity"] == "Exports"]["Volume (10^3 m3)"].sum())

In [109]:
def extract_pentanes_value(data, variable):
  return pentanes_to_pj(1000 * 365 * extract_value(data, variable))

def extract_pentanes_data(layer_1, data, year):  
  pentanes_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["pentane_imports"] = extract_pentanes_value(pentanes_data, "Net Imports")
  layer_1["pentanes_from_condensate"] = extract_pentanes_value(pentanes_data, "Production from Gas Wells - Liquid Condensate")
  layer_1["pentane_production"] = extract_pentanes_value(pentanes_data, ["Production from Refineries", "Production from Gas Processing - Pentanes Plus", "Production from Gas Wells - Liquid Condensate"])
  layer_1["pentane_exports"] = extract_pentanes_value(pentanes_data, "Net Exports")
  layer_1["pentane_diluent_demand"] = extract_pentanes_value(pentanes_data, "Diluent Demand")

  layer_1["pentane_adjustments"] = extract_pentanes_value(pentanes_data, "Total Supply - Total Disposition")

In [110]:
def extract_propane_value(data, variable):
  return propane_to_pj(1000 * 365 * extract_value(data, variable))

def extract_propane_data(layer_1, data, year):
  propane_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Unit"] == "Thousand Cubic Metres per day")]

  layer_1["propane_imports"] = extract_propane_value(propane_data, "Imports")
  layer_1["propane_from_oil_sands"] = extract_propane_value(propane_data, "Production from Oil Sands off-gas")
  layer_1["propane_production"] = extract_propane_value(propane_data, ["Production from Gas Processing", "Production from Oil Sands off-gas", "Production from Refineries"])
  layer_1["propane_exports"] = extract_propane_value(propane_data, "Projected Exports")

  layer_1["propane_oil_related_disposition"] = extract_propane_value(propane_data, ["AB Petrochemical Demand", "ON Petrochemical Demand", "Oil Sands Solvent Demand"])
  layer_1["propane_other_disposition"] = extract_propane_value(propane_data, ["AB Other Demand", "Other Canada Demand"])

  layer_1["propane_adjustments"] = extract_propane_value(propane_data, "Total Supply - Total Disposition")

In [111]:
def extract_uranium_data(layer_1, primary_demand_data, year):
  electric_generation_primary_demand_data = primary_demand_data[(primary_demand_data["Scenario"] == "Reference") & (primary_demand_data["Year"] == year) & (primary_demand_data["Region"] == "Canada") & (primary_demand_data["Sector"] == "Electric Generation")]
  # Uranium
  # -------
  # Sources
  # 1. https://www.world-nuclear.org/information-library/country-profiles/countries-a-f/canada-uranium.aspx
  # 2. https://www.cameco.com/invest/financial-information/annual-reports/2020
  # 3. https://www.cameco.com/businesses/uranium-operations/canada/cigar-lake
  # 4. https://www.nrcan.gc.ca/energy/energy-sources-distribution/uranium-nuclear-energy/uranium-canada/
  # 5. https://www.nrcan.gc.ca/energy/energy-sources-distribution/uranium-nuclear-energy/uranium-canada/about-uranium/7695
  # 6. https://www.cameco.com/invest/overview
  # 7. https://www.world-nuclear.org/information-library/nuclear-fuel-cycle/mining-of-uranium/world-uranium-mining-production.aspx
  # 
  # We have limited data here, but the totals per year are: (tonnes U3O8) [1]
  # 2019: 6,938 [7]
  # 2018: 7,001
  # 2017: 13,116
  # 2016: 14,039
  # 2015: 13,325
  # 2014: 9,134
  # 2013: 9,331

  # However if we look at Cameco (owner of Canada's Cigar Lake Uranium facility):
  # - Cameco represents 50% ownership of the Cigar Lake facility and their 2020 share was 5,000,000 pounds -- thus we can estimate that the Cigar Lake production was 10m lbs. [3]
  # - 10m lbs = 5000 tons
  # - Cigar Lake represented 100% of Canada's Uranium production in 2019. [1]
  uranium_production_2020 = 5000

  layer_1["uranium_imports"] = 0
  layer_1["uranium_production"] = uranium_to_pj(uranium_production_2020) #tons
  layer_1["uranium_used"] = extract_value(electric_generation_primary_demand_data, "Nuclear")

  # This model assumes a perfect USE or SELL world. This is unlikely to be the case, as Canada likely maintains a store.
  layer_1["uranium_exports"] = layer_1["uranium_production"] - layer_1["uranium_used"]

  # We don't need to cover this number, since it may actually represent previous-year stores being used as well
  # [6] https://www.cameco.com/invest/overview (Blind River)
  # layer_1["uranium_processing"] = 11.7m kgU

## End Use Data
Canada Energy Regulator. Canada's Energy Future Data Appendices.
DOI: https://doi.org/10.35002/zjr8-8x75

> End-use (or secondary) energy demand includes energy used in the residential, commercial, industrial and transportation sectors. This includes includes non-energy use and producer consumption. Non-energy use is the use of energy commodities for a purpose other than fuel. Examples of this include energy commodities used as petrochemical feedstock, lubricants and asphalt. Producer consumption accounts for the energy consumed by energy producers for their activity. For example, this would include the combustion of natural gas by natural gas producers to operate compressors and processing equipment.

> Commercial sector energy use includes energy used by pipelines. RPP is Refined Petroleum Products. RPP includes asphalt, aviation fuel, diesel, gasoline, heavy fuel oil, kerosene, light fuel oil, liquified petroleum gases (LPG), lubricants, naphtha specialties, oil, non-energy products, petroleum feedstocks, petroleum coke, still gas.

> Biofuels and Emerging Energy includes biomass (wood), solar, geothermal, hydrogen, ethanol and biodiesel.

> Other includes coal, coke, and coke oven gas, unless otherwise listed.

> Industrial includes energy used for oil and gas production, processing and refining. 

In [112]:
def extract_end_use_data(layer, data, year):
  # No conversion is required for this data as it is all in PJ
  end_use_data = data[(data["Scenario"] == "Reference") & (data["Year"] == year) & (data["Region"] == "Canada")]
  end_use_residential = end_use_data[end_use_data["Sector"] == "Residential"]
  layer["electricity_to_residential"] = extract_value(end_use_residential, "Electricity")
  layer["natural_gas_to_residential"] = extract_value(end_use_residential, "Natural Gas")
  layer["rpp_to_residential"] = extract_value(end_use_residential, "RPP")
  layer["biofuels_to_residential"] = extract_value(end_use_residential, "Biofuels & Emerging Energy")
  layer["other_to_residential"] = extract_value(end_use_residential, "Other")

  end_use_commercial = end_use_data[end_use_data["Sector"] == "Commercial"]
  layer["electricity_to_commercial"] = extract_value(end_use_commercial, "Electricity")
  layer["natural_gas_to_commercial"] = extract_value(end_use_commercial, "Natural Gas")
  layer["rpp_to_commercial"] = extract_value(end_use_commercial, "RPP")
  layer["biofuels_to_commercial"] = extract_value(end_use_commercial, "Biofuels & Emerging Energy")
  layer["other_to_commercial"] = extract_value(end_use_commercial, "Other")

  end_use_industrial = end_use_data[end_use_data["Sector"] == "Industrial"]
  layer["electricity_to_industrial"] = extract_value(end_use_industrial, "Electricity")
  layer["natural_gas_to_industrial"] = extract_value(end_use_industrial, "Natural Gas")
  layer["rpp_to_industrial"] = extract_value(end_use_industrial, "RPP")
  layer["biofuels_to_industrial"] = extract_value(end_use_industrial, "Biofuels & Emerging Energy")
  layer["other_to_industrial"] = extract_value(end_use_industrial, "Other")

  end_use_transportation = end_use_data[end_use_data["Sector"] == "Transportation"]
  layer["aviation_fuel_to_transportation"] = extract_value(end_use_transportation, "Aviation Fuel")
  layer["diesel_to_transportation"] = extract_value(end_use_transportation, "Diesel")
  layer["electricity_to_transportation"] = extract_value(end_use_transportation, "Electricity")
  layer["biofuels_to_transportation"] = extract_value(end_use_transportation, "Biofuels")
  layer["heavy_fuel_oil_to_transportation"] = extract_value(end_use_transportation, "Heavy Fuel Oil")
  layer["lpg_to_transportation"] = extract_value(end_use_transportation, "LPG")
  layer["lubricants_to_transportation"] = extract_value(end_use_transportation, "Lubricants")
  layer["motor_gasoline_to_transportation"] = extract_value(end_use_transportation, "Motor Gasoline")
  layer["natural_gas_to_transportation"] = extract_value(end_use_transportation, "Natural Gas")

  layer["total_electricity_end_use_demand"] = extract_value(end_use_data[end_use_data["Sector"] == "Total End-Use"], "Electricity")

In [113]:
# These helper methods allow us to pull everything together

def extract_production(layer, year):
  extract_butane_data(layer, butane_data, year)
  extract_coal_data(layer, coal_data, year)
  extract_crude_oil_data(layer, crude_oil_production_data, crude_oil_export_data, cer_imports_exports, year)
  extract_ethane_data(layer, ethane_data, year)
  extract_natural_gas_data(layer, natural_gas_data, natural_gas_export_data, year)
  extract_pentanes_data(layer, pentanes_data, year)
  extract_propane_data(layer, propane_data, year)
  extract_uranium_data(layer, primary_demand_data, year)

def extract_generation(layer, year):
  extract_electricity_data(layer, electricity_data, primary_demand_data, electricity_interchange_data, year)

def extract_end_use(layer, year):
  extract_end_use_data(layer, end_use_demand_data, year)

extract_production(layer_1_production_and_imports, 2020)
extract_generation(layer_2_primary_energy, 2020)
extract_end_use(layer_3_industry, 2020)

# Show the raw numbers

display(layer_1_production_and_imports)
display(layer_2_primary_energy)
display(layer_3_industry)

{'butane_imports': 11.040577262514,
 'butane_from_oil_sands': 6.9157953070074,
 'butane_production': 295.9841791588518,
 'butane_exports': 80.0177350380561,
 'butane_oil_related_disposition': 213.9192406007604,
 'butane_canada_other_disposition': 13.087780803441898,
 'butane_adjustments': 0.0,
 'coal_imports': 162.66573448320003,
 'coal_production': 1245.751417116,
 'coal_exports': 896.6025652440001,
 'crude_oil_imports': 1294.948605391703,
 'crude_oil_field_condensate': 648.631828098819,
 'crude_oil_production': 10279.083898502427,
 'crude_oil_exports': 8582.43694724221,
 'ethane_imports': 115.049979182862,
 'ethane_from_oil_sands': 20.299600600646404,
 'ethane_production': 238.8149581532184,
 'ethane_exports': 0.0,
 'ethane_oil_related_disposition': 353.8649373159762,
 'ethane_adjustments': 0.0,
 'natural_gas_imports': 846.7372125477906,
 'natural_gas_production': 6062.15880664835,
 'natural_gas_exports': 2645.311362001885,
 'pentane_imports': 395.79014831042747,
 'pentanes_from_cond

{'coke_and_coal_electric_generation_demand': 382.4786,
 'hydro_electric_generation_demand': 1385.126,
 'natural_gas_electric_generation_demand': 735.218,
 'uranium_electric_generation_demand': 986.3109,
 'oil_electric_generation_demand': 47.664,
 'other_renewables_electric_generation_demand': 231.8089,
 'coal_and_coke': 121.35540096,
 'hydro_wave_tidal': 1385.72064,
 'natural_gas': 252.602424,
 'uranium': 320.65128,
 'oil': 9.283500360000001,
 'biomass_geothermal': 29.512054799999998,
 'solar': 11.438586,
 'wind': 124.822944,
 'total_electricity_generation': 2255.38683012,
 'coke_and_coal_loss': 261.12319904,
 'hydro_loss': -0.5946400000000267,
 'natural_gas_loss': 482.6155759999999,
 'uranium_loss': 665.6596199999999,
 'other_renewables_loss': 66.03531520000001,
 'oil_loss': 38.38049964,
 'electricity_imports': 11.9165724,
 'electricity_exports': 244.96984799999996,
 'total_electricity_primary_demand': 3768.6064,
 'total_electricity_losses': 1513.2195698799997}

{'electricity_to_residential': 688.3434,
 'natural_gas_to_residential': 816.7201,
 'rpp_to_residential': 76.2924,
 'biofuels_to_residential': 179.6216,
 'other_to_residential': 0.0001,
 'electricity_to_commercial': 397.8073,
 'natural_gas_to_commercial': 628.1063,
 'rpp_to_commercial': 215.0534,
 'biofuels_to_commercial': 1.0926,
 'other_to_commercial': 0.9889,
 'electricity_to_industrial': 791.9285,
 'natural_gas_to_industrial': 2848.6037,
 'rpp_to_industrial': 1824.2286,
 'biofuels_to_industrial': 349.6185,
 'other_to_industrial': 119.8725,
 'aviation_fuel_to_transportation': 177.984,
 'diesel_to_transportation': 754.1136,
 'electricity_to_transportation': 4.0233,
 'biofuels_to_transportation': 109.3502,
 'heavy_fuel_oil_to_transportation': 41.6128,
 'lpg_to_transportation': 10.893,
 'lubricants_to_transportation': 4.1859,
 'motor_gasoline_to_transportation': 1355.474,
 'natural_gas_to_transportation': 6.1437,
 'total_electricity_end_use_demand': 1882.1025}

In [114]:
colours = {
  "Butane Production" : "#f94144",
  "Butane Imports" : "#f94144",
  "Butane Exports" : "#f94144",

  "Coal Production" : "#f3722c",
  "Coal Imports" : "#f3722c",
  "Primary Coal" : "#f3722c",

  "Crude Oil Production" : "#f8961e",
  "Crude Oil Imports" : "#f8961e",
  "Primary RPP" : "#f8961e",
  
  "Ethane Production" : "#f9844a",
  "Ethane Imports" : "#f9844a",
  "Ethane Exports" : "#f9844a",
  
  "Natural Gas Production" : "#f9c74f",
  "Natural Gas Imports" : "#f9c74f",
  
  "Primary Natural Gas" : "#f9c74f",
  
  "Pentane Production" : "#90be6d",
  "Pentane Imports" : "#90be6d",
  "Pentane Exports" : "#90be6d",
  
  "Propane Production" : "#4d908e",
  "Propane Imports" : "#4d908e",
  "Propane Exports" : "#4d908e",

  "Uranium Production" : "#577590",
  "Uranium Imports" : "#577590",
  "Primary Uranium" : "#577590",

  "Hydroelectric Energy Production" : "#277da1",
  "Primary Hydro" : "#277da1",

  "Wind Energy Production" : "#f3722c",
  "Primary Wind" : "#f3722c",

  "TODO ???" : "black",
  "TODO ????" : "black",

  "Primary NGL" : "#f3722c",
  "Primary Other Renewables and Landfill Gas" : "black",
}

In [115]:
from sankey import Sankey
sankey = Sankey()

# Natural Gas Liquids
# Ethane
sankey.add_edge("Ethane Production", "Primary NGL", layer_1_production_and_imports["ethane_production"])
sankey.add_edge("Ethane Imports", "Primary NGL", layer_1_production_and_imports["ethane_imports"])

sankey.add_edge("Primary NGL", "Ethane Exports", layer_1_production_and_imports["ethane_exports"])
sankey.add_edge("Primary NGL", "Primary RPP", layer_1_production_and_imports["ethane_oil_related_disposition"])
sankey.add_edge("Ethane Exports", "Exports", layer_1_production_and_imports["ethane_exports"])

if layer_1_production_and_imports["ethane_adjustments"] > 0:
  sankey.add_edge("Stored Energy", layer_1_production_and_imports["ethane_adjustments"])
elif layer_1_production_and_imports["ethane_adjustments"] < 0:
  sankey.add_edge("(Ethane Adjustments)", abs(layer_1_production_and_imports["ethane_adjustments"]))

# Pentane
sankey.add_edge("Pentane Production", "Primary NGL", layer_1_production_and_imports["pentane_production"])
sankey.add_edge("Pentane Imports", "Primary NGL", layer_1_production_and_imports["pentane_imports"])

sankey.add_edge("Primary NGL", "Pentane Exports", layer_1_production_and_imports["pentane_exports"])
sankey.add_edge("Primary NGL", "Primary RPP", layer_1_production_and_imports["pentane_diluent_demand"])
sankey.add_edge("Pentane Exports", "Exports", layer_1_production_and_imports["pentane_exports"])

if layer_1_production_and_imports["pentane_adjustments"] > 0:
  sankey.add_edge("Stored Energy", layer_1_production_and_imports["pentane_adjustments"])
elif layer_1_production_and_imports["pentane_adjustments"] < 0:
  sankey.add_edge("(Pentane Adjustments)", abs(layer_1_production_and_imports["pentane_adjustments"]))

# Propane
sankey.add_edge("Propane Production", "Primary NGL", layer_1_production_and_imports["propane_production"])
sankey.add_edge("Propane Imports", "Primary NGL", layer_1_production_and_imports["propane_imports"])

sankey.add_edge("Primary NGL", "Propane Exports", layer_1_production_and_imports["propane_exports"])
sankey.add_edge("Primary NGL", "Primary RPP", layer_1_production_and_imports["propane_oil_related_disposition"])
sankey.add_edge("Primary NGL", "Other", layer_1_production_and_imports["propane_other_disposition"])
sankey.add_edge("Propane Exports", "Exports", layer_1_production_and_imports["propane_exports"])

if layer_1_production_and_imports["propane_adjustments"] > 0:
  sankey.add_edge("Stored Energy", layer_1_production_and_imports["propane_adjustments"])
elif layer_1_production_and_imports["propane_adjustments"] < 0:
  sankey.add_edge("(Propane Adjustments)", abs(layer_1_production_and_imports["propane_adjustments"]))

# Butane
sankey.add_edge("Butane Production", "Primary NGL", layer_1_production_and_imports["butane_production"])
sankey.add_edge("Butane Imports", "Primary NGL", layer_1_production_and_imports["butane_imports"])

sankey.add_edge("Primary NGL", "Butane Exports", layer_1_production_and_imports["butane_exports"])
sankey.add_edge("Primary NGL", "Primary RPP", layer_1_production_and_imports["butane_oil_related_disposition"])
sankey.add_edge("Primary NGL", "Other", layer_1_production_and_imports["butane_canada_other_disposition"])
sankey.add_edge("Butane Exports", "Exports", layer_1_production_and_imports["butane_exports"])

if layer_1_production_and_imports["butane_adjustments"] > 0:
  sankey.add_edge("Stored Energy", layer_1_production_and_imports["butane_adjustments"])
elif layer_1_production_and_imports["butane_adjustments"] < 0:
  sankey.add_edge("(Butane Adjustments)", abs(layer_1_production_and_imports["butane_adjustments"]))

# ---

# Natural Gas
sankey.add_edge("Natural Gas Production", "Primary Natural Gas", layer_1_production_and_imports["natural_gas_production"])
sankey.add_edge("Natural Gas Imports", "Primary Natural Gas", layer_1_production_and_imports["natural_gas_imports"])

sankey.add_edge("Primary Natural Gas", "Exports", layer_1_production_and_imports["natural_gas_exports"])

# Over some periods, Canada dips into its NG inventory
natural_gas_deficit = layer_1_production_and_imports["natural_gas_production"] + layer_1_production_and_imports["natural_gas_imports"] - layer_3_industry["natural_gas_to_residential"] - layer_3_industry["natural_gas_to_commercial"] - layer_3_industry["natural_gas_to_industrial"] - layer_3_industry["natural_gas_to_transportation"] - layer_1_production_and_imports["natural_gas_exports"] - layer_2_primary_energy["natural_gas_electric_generation_demand"]
if natural_gas_deficit < 0:
  sankey.add_edge("Natural Gas Reserves / LNG Imports", "Primary Natural Gas", abs(natural_gas_deficit))

# ---

# Coal
sankey.add_edge("Coal Production", "Primary Coal", layer_1_production_and_imports["coal_production"])
sankey.add_edge("Coal Imports", "Primary Coal", layer_1_production_and_imports["coal_imports"])

sankey.add_edge("Primary Coal", "Exports", layer_1_production_and_imports["coal_exports"])

# Crude Oil
sankey.add_edge("Crude Oil Production", "Primary RPP", layer_1_production_and_imports["crude_oil_production"])
sankey.add_edge("Crude Oil Imports", "Primary RPP", layer_1_production_and_imports["crude_oil_imports"])
sankey.add_edge("Primary RPP", "Exports", layer_1_production_and_imports["crude_oil_exports"])

# Uranium
sankey.add_edge("Uranium Production", "Primary Uranium", layer_1_production_and_imports["uranium_production"])
sankey.add_edge("Uranium Imports", "Primary Uranium", layer_1_production_and_imports["uranium_imports"])
sankey.add_edge("Primary Uranium", "Exports", layer_1_production_and_imports["uranium_exports"])

# Renewables
sankey.add_edge("Wind Energy Production", "Primary Wind", layer_2_primary_energy["wind"])
sankey.add_edge("Hydroelectric Energy Production", "Primary Hydro", layer_2_primary_energy["hydro_electric_generation_demand"])
sankey.add_edge("Solar Energy Production", "Primary Solar", layer_2_primary_energy["solar"])

#TODO: This needs to be properly calculated
sankey.add_edge("Biomass & Geothermal", "Primary Other Renewables and Landfill Gas", layer_2_primary_energy["biomass_geothermal"])

# Other
#TODO: # Hardcoded value to balance
sankey.add_edge("TODO ???", "TODO ????", 705.7182151999999)
sankey.add_edge("TODO ????", "Primary Other Renewables and Landfill Gas", 705.7182151999999)

# Electricity
sankey.add_edge("Electricity Imports", "Produced and Distributed Electricity", layer_2_primary_energy["electricity_imports"])
sankey.add_edge("Primary Coal", "Electricity Generation", layer_2_primary_energy["coke_and_coal_electric_generation_demand"])
sankey.add_edge("Primary Hydro", "Electricity Generation", layer_2_primary_energy["hydro_electric_generation_demand"])
sankey.add_edge("Primary Natural Gas", "Electricity Generation", layer_2_primary_energy["natural_gas_electric_generation_demand"])
sankey.add_edge("Primary Uranium", "Electricity Generation", layer_2_primary_energy["uranium_electric_generation_demand"])
sankey.add_edge("Primary RPP", "Electricity Generation", layer_2_primary_energy["oil_electric_generation_demand"])
# We subtract solar and wind here to ensure we're not double counting
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Electricity Generation", layer_2_primary_energy["other_renewables_electric_generation_demand"] - layer_2_primary_energy["solar"] - layer_2_primary_energy["wind"])
sankey.add_edge("Primary Solar", "Electricity Generation", layer_2_primary_energy["solar"])
sankey.add_edge("Primary Wind", "Electricity Generation", layer_2_primary_energy["wind"])

sankey.add_edge("Primary Natural Gas", "Residential", layer_3_industry["natural_gas_to_residential"])
sankey.add_edge("Primary Natural Gas", "Commercial", layer_3_industry["natural_gas_to_commercial"])
sankey.add_edge("Primary Natural Gas", "Industrial", layer_3_industry["natural_gas_to_industrial"])
sankey.add_edge("Primary Natural Gas", "Transportation", layer_3_industry["natural_gas_to_transportation"])

sankey.add_edge("Primary RPP", "Residential", layer_3_industry["rpp_to_residential"])
sankey.add_edge("Primary RPP", "Commercial", layer_3_industry["rpp_to_commercial"])
sankey.add_edge("Primary RPP", "Industrial", layer_3_industry["rpp_to_industrial"])
sankey.add_edge("Primary RPP", "Transportation", layer_3_industry["aviation_fuel_to_transportation"] + layer_3_industry["diesel_to_transportation"] + layer_3_industry["heavy_fuel_oil_to_transportation"] + layer_3_industry["lpg_to_transportation"] + layer_3_industry["lubricants_to_transportation"] + layer_3_industry["motor_gasoline_to_transportation"])

#TODO: This may be misclassified since this is "Biofuels and Emerging Energy" (the full definition is 'Biomass (wood), solar, geothermal, hydrogen, ethanol and biodiesel.')
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Residential", layer_3_industry["biofuels_to_residential"])
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Commercial", layer_3_industry["biofuels_to_commercial"])
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Industrial", layer_3_industry["biofuels_to_industrial"])
sankey.add_edge("Primary Other Renewables and Landfill Gas", "Transportation", layer_3_industry["biofuels_to_transportation"])

# Other is defined as "Coal, coke, coke oven gas and steam"
sankey.add_edge("Primary Coal", "Residential", layer_3_industry["other_to_residential"])
sankey.add_edge("Primary Coal", "Commercial", layer_3_industry["other_to_commercial"])
sankey.add_edge("Primary Coal", "Industrial", layer_3_industry["other_to_industrial"])

coal_remainder = layer_1_production_and_imports["coal_production"] + layer_1_production_and_imports["coal_imports"] - layer_2_primary_energy["coke_and_coal_electric_generation_demand"] - layer_1_production_and_imports["coal_exports"] - layer_3_industry["other_to_residential"] - layer_3_industry["other_to_commercial"] - layer_3_industry["other_to_industrial"]

sankey.add_edge("Primary Coal", "Non-Energy Uses", coal_remainder)


# Electricity Generation & Losses

sankey.add_edge("Electricity Generation", "Generated Hydro", layer_2_primary_energy["hydro_wave_tidal"])
sankey.add_edge("Electricity Generation", "Generated Nuclear", layer_2_primary_energy["uranium"])
sankey.add_edge("Electricity Generation", "Generated Natural Gas", layer_2_primary_energy["natural_gas"])
sankey.add_edge("Electricity Generation", "Generated Oil", layer_2_primary_energy["oil"])
sankey.add_edge("Electricity Generation", "Generated Biomass & Geothermal", layer_2_primary_energy["biomass_geothermal"])
sankey.add_edge("Electricity Generation", "Generated Solar", layer_2_primary_energy["solar"])
sankey.add_edge("Electricity Generation", "Generated Wind", layer_2_primary_energy["wind"])
sankey.add_edge("Electricity Generation", "Generated Coal & Coke", layer_2_primary_energy["coal_and_coke"])

sankey.add_edge("Generated Hydro", "Produced and Distributed Electricity", layer_2_primary_energy["hydro_wave_tidal"])
sankey.add_edge("Generated Nuclear", "Produced and Distributed Electricity", layer_2_primary_energy["uranium"])
sankey.add_edge("Generated Natural Gas", "Produced and Distributed Electricity", layer_2_primary_energy["natural_gas"])
sankey.add_edge("Generated Oil", "Produced and Distributed Electricity", layer_2_primary_energy["oil"])
sankey.add_edge("Generated Biomass & Geothermal", "Produced and Distributed Electricity", layer_2_primary_energy["biomass_geothermal"])
sankey.add_edge("Generated Solar", "Produced and Distributed Electricity", layer_2_primary_energy["solar"])
sankey.add_edge("Generated Wind", "Produced and Distributed Electricity", layer_2_primary_energy["wind"])
sankey.add_edge("Generated Coal & Coke", "Produced and Distributed Electricity", layer_2_primary_energy["coal_and_coke"])

sankey.add_edge("Electricity Generation", "Hydro Generation Losses", layer_2_primary_energy["hydro_loss"])
sankey.add_edge("Electricity Generation", "Nuclear Generation Losses", layer_2_primary_energy["uranium_loss"])
sankey.add_edge("Electricity Generation", "Natural Gas Generation Losses", layer_2_primary_energy["natural_gas_loss"])
sankey.add_edge("Electricity Generation", "Oil Generation Losses", layer_2_primary_energy["oil_loss"])
sankey.add_edge("Electricity Generation", "Other Renewable Generation Losses", layer_2_primary_energy["other_renewables_loss"])
sankey.add_edge("Electricity Generation", "Coal & Coke Generation Losses", layer_2_primary_energy["coke_and_coal_loss"])

# Secondary Energy Demand

sankey.add_edge("Produced and Distributed Electricity", "Exports", layer_2_primary_energy["electricity_exports"])
sankey.add_edge("Produced and Distributed Electricity", "Residential", layer_3_industry["electricity_to_residential"])
sankey.add_edge("Produced and Distributed Electricity", "Commercial", layer_3_industry["electricity_to_commercial"])
sankey.add_edge("Produced and Distributed Electricity", "Industrial", layer_3_industry["electricity_to_industrial"])
sankey.add_edge("Produced and Distributed Electricity", "Transportation", layer_3_industry["electricity_to_transportation"])

# Could be transmission, could be unused. TODO.
misc_electricity_losses = layer_2_primary_energy["total_electricity_primary_demand"] + layer_2_primary_energy["electricity_imports"] - layer_2_primary_energy["total_electricity_losses"] - layer_3_industry["total_electricity_end_use_demand"] - layer_2_primary_energy["electricity_exports"]
sankey.add_edge("Produced and Distributed Electricity", "Misc. Losses", misc_electricity_losses)

sankey.add_edge("Misc. Losses", "Losses", misc_electricity_losses)
sankey.add_edge("Coal & Coke Generation Losses", "Losses", layer_2_primary_energy["coke_and_coal_loss"])
sankey.add_edge("Hydro Generation Losses", "Losses", layer_2_primary_energy["hydro_loss"])
sankey.add_edge("Nuclear Generation Losses", "Losses", layer_2_primary_energy["uranium_loss"])
sankey.add_edge("Natural Gas Generation Losses", "Losses", layer_2_primary_energy["natural_gas_loss"])
sankey.add_edge("Oil Generation Losses", "Losses", layer_2_primary_energy["oil_loss"])
sankey.add_edge("Other Renewable Generation Losses", "Losses", layer_2_primary_energy["other_renewables_loss"])

sankey.set_colours(colours)

sankey.render("2020 Canadian Energy Future Sankey Diagram")


In [116]:
# Validate the values. This will print out all nodes that have an unbalanced value (aka more in than out, or more out than in, if not an edge node.)
sankey.validate()

Primary RPP is unbalanced: 492.5750269692153


In [117]:
# Scratch pad area -- This is just various experiments. You can ignore everything after this. ----

# To validate that our data is more or less correct, we're looking at multiple sources. This is a comparison of electricity generation between Stats Can and CER's Energy Futures. Stats Can is likely more accurate.
total_mwh_2020 = statscan_electricity_data_2020[statscan_electricity_data_2020["Type of electricity generation"] == "Total all types of electricity generation"]["VALUE"].sum()
nuclear_mwh_2020 = statscan_electricity_data_2020[statscan_electricity_data_2020["Type of electricity generation"] == "Nuclear steam turbine"]["VALUE"].sum()
total_pj_generated_electricity = total_mwh_2020 * mwh_to_pj

print(f'Canada generated {total_pj_generated_electricity} PJ (Statscan) vs {layer_2_primary_energy["total_electricity_generation"]} PJ (CER).')
print(f'Canada generated {nuclear_mwh_2020 * mwh_to_pj} PJ via Nuclear (Statscan) vs {layer_2_primary_energy["uranium"]} PJ via Nuclear (CER).')


Canada generated 2288.0595599999997 PJ (Statscan) vs 2255.38683012 PJ (CER).
Canada generated 333.546678 PJ via Nuclear (Statscan) vs 320.65128 PJ via Nuclear (CER).


In [118]:
layer_1_production_and_imports["crude_oil_field_condensate"]

648.631828098819

In [119]:
# TODO: We ignore this value (crude_oil_field_condensate) as it is roughly a sum of the following values:
display(layer_1_production_and_imports["crude_oil_field_condensate"])

# TODO: Verify with an expert if this is correct.
display(layer_1_production_and_imports["butane_from_oil_sands"])
display(layer_1_production_and_imports["ethane_from_oil_sands"])
display(layer_1_production_and_imports["propane_from_oil_sands"])
display(layer_1_production_and_imports["pentanes_from_condensate"])

# Natural Gas -- These are a bit lower than the Reference Case but within the same ballpark.

# http://www.cer-rec.gc.ca/en/data-analysis/energy-commodities/natural-gas/statistics/marketable-natural-gas-production-in-canada.html
# Marketable Production (10^3 m^3 / d)
# Daily Average 436875 10^3 m^3 / d
ng_production = natural_gas_to_pj(436875 * 1000 * 365)
display(f'NG Production: {ng_production}')

# Source Annual trade summary: http://www.cer-rec.gc.ca/en/data-analysis/energy-commodities/natural-gas/report/natural-gas-summary/natural-gas-annual-trade-summary.html
ng_exports = natural_gas_to_pj(70900000000)
ng_imports = natural_gas_to_pj(22700000000)

ng_domestic_demand = natural_gas_to_pj(378 * 1000000 * 365)
display(ng_exports)
display(ng_imports)
display(ng_domestic_demand)

# Canada shows an inventory deficit of NG YoY: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2510005501&pickMembers%5B0%5D=1.1&pickMembers%5B1%5D=3.1&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2020&cubeTimeFrame.endMonth=12&cubeTimeFrame.endYear=2020&referencePeriods=20200101%2C20201201
display(ng_production + ng_imports - ng_exports - ng_domestic_demand)



648.631828098819

6.9157953070074

20.299600600646404

14.542557128024399

735.7993854633321

'NG Production: 5947.8346875'

2644.57

846.71

5146.281

-996.3063124999999